# word2vec implementation

## Overview
<img src="image/word2vec_cbow_mechanism.png" align="left"/>

---
# Setups

In [1]:
import sys
import os
import re
from itertools import islice
from typing import Dict, List
import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=400) 

## Setup for Google Colab environment

In [2]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/github
    !mkdir -p /content/github
    !git clone https://github.com/oonisim/python-programs.git /content/github
        
    import sys
    sys.path.append('/content/github/nlp/src')

## Jupyter notebook setups

In [3]:
%load_ext line_profiler
%load_ext autoreload

## Utilites

In [4]:
%autoreload 2

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

import function.fileio as fileio

# Constants

In [5]:
USE_PTB = True
DEBUG = False
VALIDATION = True

---

# Data
## Corpus

In [6]:
corpus = "To be, or not to be, that is the question that matters"
_file = "ptb.train.txt"
if USE_PTB:
    if not fileio.Function.is_file(f"~/.keras/datasets/{_file}"):
        path_to_ptb = tf.keras.utils.get_file(
            _file, 
            f'https://raw.githubusercontent.com/tomsercu/lstm/master/data/{_file}'
        )
    corpus = fileio.Function.read_file(path_to_ptb)

In [7]:
examples = corpus.split('\n')[:5]
for line in examples:
    print(line)

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate 
 a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported 


---
# Word indexing

In [8]:
%autoreload 2
from layer.preprocessing import (
    WordIndexing, 
    EventContext
)

## WordIndexing instance for the corpus

Adapt to the ```corpus``` and provides:
* word_to_index dictionary
* vocaburary of the corpus
* word occurrence probabilites

In [9]:
word_indexing = WordIndexing(
    name="word_indexing_on_ptb",
    corpus=corpus
)

In [10]:
words = word_indexing.list_words(range(10))
print(f"WordIndexing.vocabulary[10]:\n{words}\n")

indices = word_indexing.list_word_indices(words)
print(f"WordIndexing.word_to_index[10]:")
for item in zip(words, indices):
    print(item)

probabilities = word_indexing.list_probabilities(words)
print(f"\nWordIndexing.probabilities[10]:")
for word, p in zip(words, probabilities):
    print(f"{word:20s} : {p:.5e}")

WordIndexing.vocabulary[10]:
['<nil>' '<unk>' 'knight' 'sequester' 'styles' 'thurmond' 'claimed' 'cigarette' 'ringing' 'routine']

WordIndexing.word_to_index[10]:
('<nil>', 0)
('<unk>', 1)
('knight', 2)
('sequester', 3)
('styles', 4)
('thurmond', 5)
('claimed', 6)
('cigarette', 7)
('ringing', 8)
('routine', 9)

WordIndexing.probabilities[10]:
<nil>                : 0.00000e+00
<unk>                : 5.05607e-02
knight               : 3.03229e-05
sequester            : 7.86150e-06
styles               : 1.01076e-05
thurmond             : 1.12307e-05
claimed              : 3.93075e-05
cigarette            : 2.02153e-05
ringing              : 8.98457e-06
routine              : 2.13384e-05


## Sentence to Sequence

In [11]:
# sentences = "\n".join(corpus.split('\n')[5:6])
sentences = """
the asbestos fiber <unk> is unusually <unk> once it enters the <unk> 
with even brief exposures to it causing symptoms that show up decades later researchers said
"""
sequences = word_indexing.function(sentences)
for pair in zip(sentences.strip().split(" "), sequences[0]):
    print(f"{pair[0]:15} : {pair[1]:5}")

Sentence is empty. Skipping...
Sentence is empty. Skipping...


the             :  9410
asbestos        :  8836
fiber           :   403
<unk>           :     1
is              :  5948
unusually       :  3046
<unk>           :     1
once            :  3883
it              :  4222
enters          :  5787
the             :  9410
<unk>           :     1

with           :     0
even            :     0
brief           :     0


---
# Context of a word in a sentence

In the sentence ```"a form of asbestos once used to make kent cigarette filters"```, one of the context windows ```a form of asbestos once``` of size 5 and event size 1 has.
* ```of``` as a target word.
* ```(a, form) and (asbestos, once)``` as its context.

### Sequence of the word indices for the sentence

In [12]:
sentences = """
a form of asbestos once used to make kent cigarette filters

N years old and former chairman of consolidated gold fields plc was named a nonexecutive director
"""

sequence = word_indexing.function(sentences)
sequence

Sentence is empty. Skipping...
Sentence is empty. Skipping...
Sentence is empty. Skipping...


array([[8798, 7523, 2756, 8836, 3883, 3711, 8308, 2546, 5834,    7, 3927,    0,    0,    0,    0,    0],
       [4529, 8761, 1554, 2844, 9485, 8276, 2756, 9551, 7301, 2882, 4751, 1345,   59, 8798, 2066, 6664]])

## Target, context pairs

For each word in the setence ```(of, asbestos, ... , kent)``` excludnig the ends of the sentence, create ```(target, context)``` as:

```
[
  [of, a, form, asbestos, once],              # target is 'of', context is (a, form, asbestos, once)
  ['asbestos', 'form', 'of', 'once', 'used'],
  ['once', 'of', 'asbestos', 'used', 'to'],
  ...
]
```

In [13]:
event_context = EventContext(
    name="ev",
    window_size=5,
    event_size=1    
)

In [14]:
event_context_pairs = event_context.function(sequence)
event_context_pairs

array([[[2756, 8798, 7523, 8836, 3883],
        [8836, 7523, 2756, 3883, 3711],
        [3883, 2756, 8836, 3711, 8308],
        [3711, 8836, 3883, 8308, 2546],
        [8308, 3883, 3711, 2546, 5834],
        [2546, 3711, 8308, 5834,    7],
        [5834, 8308, 2546,    7, 3927],
        [   7, 2546, 5834, 3927,    0],
        [3927, 5834,    7,    0,    0],
        [   0,    7, 3927,    0,    0],
        [   0, 3927,    0,    0,    0],
        [   0,    0,    0,    0,    0]],

       [[1554, 4529, 8761, 2844, 9485],
        [2844, 8761, 1554, 9485, 8276],
        [9485, 1554, 2844, 8276, 2756],
        [8276, 2844, 9485, 2756, 9551],
        [2756, 9485, 8276, 9551, 7301],
        [9551, 8276, 2756, 7301, 2882],
        [7301, 2756, 9551, 2882, 4751],
        [2882, 9551, 7301, 4751, 1345],
        [4751, 7301, 2882, 1345,   59],
        [1345, 2882, 4751,   59, 8798],
        [  59, 4751, 1345, 8798, 2066],
        [8798, 1345,   59, 2066, 6664]]], dtype=int32)

### Target, context pairs in textual words

In [15]:
word_indexing.sequence_to_sentence(event_context_pairs)

[[['of', 'a', 'form', 'asbestos', 'once'],
  ['asbestos', 'form', 'of', 'once', 'used'],
  ['once', 'of', 'asbestos', 'used', 'to'],
  ['used', 'asbestos', 'once', 'to', 'make'],
  ['to', 'once', 'used', 'make', 'kent'],
  ['make', 'used', 'to', 'kent', 'cigarette'],
  ['kent', 'to', 'make', 'cigarette', 'filters'],
  ['cigarette', 'make', 'kent', 'filters', '<nil>'],
  ['filters', 'kent', 'cigarette', '<nil>', '<nil>'],
  ['<nil>', 'cigarette', 'filters', '<nil>', '<nil>'],
  ['<nil>', 'filters', '<nil>', '<nil>', '<nil>'],
  ['<nil>', '<nil>', '<nil>', '<nil>', '<nil>']],
 [['old', 'n', 'years', 'and', 'former'],
  ['and', 'years', 'old', 'former', 'chairman'],
  ['former', 'old', 'and', 'chairman', 'of'],
  ['chairman', 'and', 'former', 'of', 'consolidated'],
  ['of', 'former', 'chairman', 'consolidated', 'gold'],
  ['consolidated', 'chairman', 'of', 'gold', 'fields'],
  ['gold', 'of', 'consolidated', 'fields', 'plc'],
  ['fields', 'consolidated', 'gold', 'plc', 'was'],
  ['plc', 'g